# Elasticsearch migration

## ES data dump

import json

# Load Data Dump
with open('../../data/index-dump/index_skatt_data.json', 'r') as file:
    data_dump = json.load(file)

# Load Mapping Dump
with open('../../data/index-dump/index_skatt_mapping.json', 'r') as file:
    mapping_dump = json.load(file)


## Data migration

### Local ES

In [19]:
from elasticsearch import Elasticsearch

es_local = Elasticsearch("http://localhost:9200")

In [26]:
# Define the query
query = {
    "query": {
        "match_all": {}
    },
    "size": 5  # Fetch 5 documents
}

# Replace 'your_index' with your actual index name
response = es_local.search(index="index_skatt", body=query)

# Print the results
for hit in response['hits']['hits']:
    print(hit['_source'])

{'embedding': [-0.008228981867432594, 0.006816971115767956, 0.009229900315403938, -0.012890403158962727, -0.002940198639407754, 0.01747317984700203, -0.024393819272518158, -0.014627711847424507, -0.026910414919257164, -0.019875384867191315, 0.027210690081119537, -0.004150237888097763, -0.014427527785301208, 0.0052977195009589195, -0.00896537210792303, 0.009823301807045937, 0.02008986845612526, -0.017916444689035416, 0.009151256643235683, -0.00247191172093153, -0.02728218398988247, 0.012454288080334663, 0.012540081515908241, -0.0009231687872670591, -0.0024433140642941, 0.01750177890062332, 0.010509646497666836, -0.021677039563655853, -0.0032780086621642113, -0.003328054677695036, 0.004611375275999308, -0.02716779336333275, -0.015685826539993286, -0.010874266736209393, -0.02407924458384514, -0.0029723711777478456, 0.001595214125700295, -0.036519233137369156, 0.034288614988327026, -0.02133386768400669, -0.021090786904096603, -0.0012019961141049862, 0.0015424871817231178, -0.02240628004074

In [28]:
# Define the query
query = {
    "query": {
        "match_all": {}
    },
    "size": 5  # Fetch 5 documents
}

# Replace 'your_index' with your actual index name
response = es_fly.search(index="index_skatt", body=query)

# Print the results
for hit in response['hits']['hits']:
    print(hit['_source'])

/Users/fabian.s.dietrichson/Library/Caches/pypoetry/virtualenvs/data-processing-PWXJwi7T-py3.12/lib/python3.12/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'elasticsearch-llm-spring-glitter-3589.fly.dev'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


{'embedding': [-0.008228981867432594, 0.006816971115767956, 0.009229900315403938, -0.012890403158962727, -0.002940198639407754, 0.01747317984700203, -0.024393819272518158, -0.014627711847424507, -0.026910414919257164, -0.019875384867191315, 0.027210690081119537, -0.004150237888097763, -0.014427527785301208, 0.0052977195009589195, -0.00896537210792303, 0.009823301807045937, 0.02008986845612526, -0.017916444689035416, 0.009151256643235683, -0.00247191172093153, -0.02728218398988247, 0.012454288080334663, 0.012540081515908241, -0.0009231687872670591, -0.0024433140642941, 0.01750177890062332, 0.010509646497666836, -0.021677039563655853, -0.0032780086621642113, -0.003328054677695036, 0.004611375275999308, -0.02716779336333275, -0.015685826539993286, -0.010874266736209393, -0.02407924458384514, -0.0029723711777478456, 0.001595214125700295, -0.036519233137369156, 0.034288614988327026, -0.02133386768400669, -0.021090786904096603, -0.0012019961141049862, 0.0015424871817231178, -0.02240628004074

### Fly ES

In [17]:
from elasticsearch import Elasticsearch


# Replace 'username' and 'password' with your actual Elasticsearch credentials
es_fly = Elasticsearch(
    ["https://elasticsearch-llm-spring-glitter-3589.fly.dev"],  # You can list multiple hosts in this array
    basic_auth=('elastic', 'abc'),
    verify_certs=False  # Not recommended for production
)


/Users/fabian.s.dietrichson/Library/Caches/pypoetry/virtualenvs/data-processing-PWXJwi7T-py3.12/lib/python3.12/site-packages/elasticsearch/_sync/client/__init__.py:399: SecurityWarning: Connecting to 'https://elasticsearch-llm-spring-glitter-3589.fly.dev:443' using TLS with verify_certs=False is insecure
  _transport = transport_class(


### Migrate index and data

In [24]:
def clean_settings(settings):
    # A list of settings that should not be transferred between indices
    ignored_settings = {
        "index.creation_date",
        "index.uuid",
        "index.version.created",
        "index.provided_name",
        "index.routing.allocation.initial_recovery._id"
    }

    # Remove settings that should not be transferred
    settings = {k: v for k, v in settings.items() if k not in ignored_settings}
    return settings

# Retrieve index settings and mappings from the local instance
local_index_info = es_local.indices.get(index=index_name)
settings = local_index_info[index_name]['settings']['index']
mappings = local_index_info[index_name]['mappings']

# Clean the settings to remove those that should not be transferred
settings = clean_settings(settings)

print(settings)

# Create the index on the Fly.io instance with the cleaned settings and mappings
if not es_fly.indices.exists(index=index_name):
    es_fly.indices.create(
        index=index_name,
        body={
            "settings": settings,
            "mappings": mappings
        }
    )
    print(f"Created index {index_name} on Fly.io with new settings and mappings")
else:
    print(f"Index {index_name} already exists on Fly.io")


{'routing': {'allocation': {'include': {'_tier_preference': 'data_content'}}}, 'number_of_shards': '1', 'provided_name': 'index_skatt', 'creation_date': '1713365019264', 'number_of_replicas': '1', 'uuid': 'wbyagqXEQTWjyBVfhomdeQ', 'version': {'created': '8500010'}}


/Users/fabian.s.dietrichson/Library/Caches/pypoetry/virtualenvs/data-processing-PWXJwi7T-py3.12/lib/python3.12/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'elasticsearch-llm-spring-glitter-3589.fly.dev'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/Users/fabian.s.dietrichson/Library/Caches/pypoetry/virtualenvs/data-processing-PWXJwi7T-py3.12/lib/python3.12/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'elasticsearch-llm-spring-glitter-3589.fly.dev'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


BadRequestError: BadRequestError(400, 'illegal_argument_exception', 'unknown setting [index.creation_date] please check that any required plugins are installed, or check the breaking changes documentation for removed settings')

In [29]:
# Index name to migrate
index_name = "index_skatteparagraf"

# Check if the index exists on Fly.io and delete it if it does
if es_fly.indices.exists(index=index_name):
    es_fly.indices.delete(index=index_name)
    print(f"Deleted existing index {index_name} on Fly.io")


# Retrieve index settings and mappings from the local instance
local_index_info = es_local.indices.get(index=index_name)
settings = local_index_info[index_name]['settings']['index']
mappings = local_index_info[index_name]['mappings']

# Create the same index on the Fly.io instance
es_fly.indices.create(
    index=index_name,
    body={
        'mappings': mappings
    }
)

/Users/fabian.s.dietrichson/Library/Caches/pypoetry/virtualenvs/data-processing-PWXJwi7T-py3.12/lib/python3.12/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'elasticsearch-llm-spring-glitter-3589.fly.dev'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/Users/fabian.s.dietrichson/Library/Caches/pypoetry/virtualenvs/data-processing-PWXJwi7T-py3.12/lib/python3.12/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'elasticsearch-llm-spring-glitter-3589.fly.dev'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'index_skatteparagraf'})

In [30]:
from elasticsearch.helpers import scan, bulk

# Function to fetch data from the local instance
def fetch_data(es_client, index):
    return scan(es_client, query={"query": {"match_all": {}}}, index=index)

# Function to transfer data to the Fly.io instance
def transfer_data(local_data, es_client_target, index):
    actions = ({
        "_index": index,
        "_id": doc['_id'],
        "_source": doc['_source']
    } for doc in local_data)

    # Use bulk helper to transfer data
    bulk(es_client_target, actions)

# Fetch data from local instance
data_to_transfer = fetch_data(es_local, index_name)

# Transfer data to Fly.io instance
transfer_data(data_to_transfer, es_fly, index_name)


/Users/fabian.s.dietrichson/Library/Caches/pypoetry/virtualenvs/data-processing-PWXJwi7T-py3.12/lib/python3.12/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'elasticsearch-llm-spring-glitter-3589.fly.dev'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/Users/fabian.s.dietrichson/Library/Caches/pypoetry/virtualenvs/data-processing-PWXJwi7T-py3.12/lib/python3.12/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'elasticsearch-llm-spring-glitter-3589.fly.dev'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/Users/fabian.s.dietrichson/Library/Caches/pypoetry/virtualenvs/data-processing-PWXJwi7T-py3.12/lib/python3.12/site-packages/urllib3/connectionpool.

In [ ]:
from elasticsearch.helpers import scan, bulk

# Function to fetch data from the local instance
def fetch_data(es_client, index):
    return scan(es_client, query={"query": {"match_all": {}}}, index=index)

# Function to transfer data to the Fly.io instance
def transfer_data(local_data, es_client_target, index):
    actions = ({
        "_index": index,
        "_id": doc['_id'],
        "_source": doc['_source']
    } for doc in local_data)

    # Use bulk helper to transfer data
    bulk(es_client_target, actions)

# Fetch data from local instance
data_to_transfer = fetch_data(es_local, index_name)

# Transfer data to Fly.io instance
transfer_data(data_to_transfer, es_fly, index_name)


In [9]:
# Create index in FLy
index_mappings = {
    "mappings": {
        "properties": {
            "content": {"type": "text"},
            "embedding": {"type": "dense_vector", "dims": 1536, "index": True, "similarity": "cosine"},
            "metadata": {
                "properties": {
                    "_node_content": {"type": "text", "fields": {"keyword": {"type": "keyword", "ignore_above": 256}}},
                    "_node_type": {"type": "text", "fields": {"keyword": {"type": "keyword", "ignore_above": 256}}},
                    "creation_date": {"type": "date"},
                    "doc_id": {"type": "keyword"},
                    "document_id": {"type": "keyword"},
                    "file_name": {"type": "text", "fields": {"keyword": {"type": "keyword", "ignore_above": 256}}},
                    "file_path": {"type": "text", "fields": {"keyword": {"type": "keyword", "ignore_above": 256}}},
                    "file_size": {"type": "long"},
                    "file_type": {"type": "text", "fields": {"keyword": {"type": "keyword", "ignore_above": 256}}},
                    "last_modified_date": {"type": "date"},
                    "page_label": {"type": "text", "fields": {"keyword": {"type": "keyword", "ignore_above": 256}}},
                    "ref_doc_id": {"type": "keyword"}
                }
            }
        }
    }
}

# Create the index with the defined mappings
if not es_fly.indices.exists(index='index_skatt'):
    es_fly.indices.create(index='index_skatt', body=index_mappings)
    print("Index created")


Index created


In [18]:
import json
from elasticsearch import Elasticsearch, helpers

# Function to read newline-delimited JSON file
def load_ndjson_file(file_path):
    with open(file_path, 'r') as file:
        for line in file:
            yield json.loads(line)

# Bulk index the JSON data
def bulk_index_documents(docs):
    actions = [
        {
            "_index": "index_skatt",  # Ensure this matches your index name
            "_source": doc
        } 
        for doc in docs
    ]

    
    try:
        successes, _ = helpers.bulk(es_fly, actions)
        print(f"Bulk indexed {successes} documents successfully.")
    except Exception as e:
        print(f"Error in bulk indexing: {str(e)}")

file_path = '../../data/index-dump/index_skatt_data.json'

# Load data from NDJSON file and index
documents = load_ndjson_file(file_path)
bulk_index_documents(documents)


/Users/fabian.s.dietrichson/Library/Caches/pypoetry/virtualenvs/data-processing-PWXJwi7T-py3.12/lib/python3.12/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'elasticsearch-llm-spring-glitter-3589.fly.dev'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Error in bulk indexing: 500 document(s) failed to index.
